In [1]:
import requests
import json
import os
import re
import datetime
import time
import string
import pandas as pd
import urllib
import random
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
import yfinance as yf
from time import sleep
from bs4 import BeautifulSoup
from requests import get
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from transformers import pipeline
from bs4 import BeautifulSoup as bs
from joblib import Parallel, delayed

C:\Users\endyj\anaconda3\envs\py38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Google Scraping

In [2]:
def get_useragent():
    return random.choice(_useragent_list)

_useragent_list = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36 Edg/111.0.1661.62',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/111.0'
]

request_timeout = 10
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'Connection': 'keep-alive',
        'Cache-Control': 'private, max-age=0',
}
html_pattern = "<(?:\"[^\"]*\"['\"]*|'[^']*'['\"]*|[^'\">])+>"
TODAY = datetime.date.today()
#TODAY = '2024-03-22'

#search variables
from_date = '1/1/2024'
to_date = '1/3/2024'
total_news = 20
sleep_interval = 20
stock_name = 'NVDA'

#NLP variables
model_id = "mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
sentiment_pipe = pipeline("sentiment-analysis", model=model_id)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [3]:
"""googlesearch is a Python library for searching Google, easily."""
def get_headers(): 
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:49.0) Gecko/20100101 Firefox/49.0',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.5',
        'Accept-Encoding': 'gzip, deflate',
        'Connection': 'keep-alive',
        'Cache-Control': 'private, max-age=0',
    }

    return headers

def _req(term, results, lang, start, proxies, timeout, from_date, to_date):
    resp = get(
        url="https://www.google.com/search",
        headers={
            "User-Agent": get_useragent()
        },
        params={
            "q": term,
            "num": results,  # Prevents multiple requests
            "hl": lang,
            "start": start,
            "lr": "lang_en",
            "tbm": 'nws',
            "tbs": f'sbd:1,cdr:1,cd_min:{from_date},cd_max:{to_date}'
            #"tbs": "qdr:d"
            #"tbs": f"sbd:1,lr:lang_en,cdr:1,cd_min:1/1/{from_year},cd_max:{to_year}"
        },
        proxies=proxies,
        timeout=timeout,
    )
    resp.raise_for_status()
    return resp

def search(term, num_results=10, lang="en", proxy=None, advanced=False, sleep_interval=0, timeout=5, from_date='1/1/2020', to_date='1/1/2020'):
    """Search the Google search engine"""

    escaped_term = urllib.parse.quote_plus(term) # make 'site:xxx.xxx.xxx ' works.

    # Proxy
    proxies = None
    if proxy:
        if proxy.startswith("https"):
            proxies = {"https": proxy}
        else:
            proxies = {"http": proxy}

    # Fetch
    start = 0
    res = []
    while start < num_results:
        # Send request
        resp = _req(escaped_term, num_results - start,
                    lang, start, proxies, timeout, from_date, to_date)
        # Parse
        soup = BeautifulSoup(resp.text, "html.parser")
        result_block = soup.find_all("div", attrs={"class": "SoaBEf"})

        if len(result_block)==0:
            break

        for result in result_block:
            # Find url, title, description
            url = result.find("a", href=True)
            title = result.find("div", attrs={"class": "n0jPhd ynAwRc MBeuO nDgy9d"})
            date = result.find("div", attrs={"class": "OSrXXb rbYSKb LfVVr"})
            obj = {}
            obj["title"] = None
            obj["date"] = None
            obj["url"] = None
            start += 1

            if title:
                obj["title"] = title.text
            if date:
                obj["date"] = from_date
            if url:
                obj["url"] = url["href"]
                res.append(obj)
                        
        sleep(sleep_interval)
    return res

In [4]:
def get_google_news_urls(q, total, from_date, to_date, sleep_interval=0):
    urls = []
    urls = search(q, num_results=total, from_date=from_date, to_date=to_date, sleep_interval=sleep_interval)
    return urls

In [5]:
def save_news_urls(q, total, from_date, to_date, temp_save=False):
    dir_name = str(TODAY)
    if dir_name not in os.listdir():
        cwd = os.getcwd()
        os.mkdir(f'{cwd}/{dir_name}')
        
    urls = get_google_news_urls(q, total, from_date, to_date, sleep_interval = 10)
    # temp
    if temp_save:
        # replace date format to file savable characters
        filename = f'{query}_{from_date.replace("/", "_")}.json'
        f = open(f'./{TODAY}/{filename}', 'w')
        f.write(json.dumps(json_object, indent = 4))
        f.close()
    return urls

In [6]:
def get_news_data(url):
    text = ""
    try:
        article = requests.get(
            url,
            headers=headers,
            timeout=request_timeout
        )
        status_code = article.status_code
        if status_code >= 400:
            f = open(f'./{TODAY}/ERROR_LOG.txt', 'a')
            f.write(f'[{time.asctime(time.localtime())}] Code {status_code}: {url}\n')
            f.close()
            return None
        soup = bs(article.content, "html.parser")
        article_body = soup.find("body")
        paragraphs = article_body.find_all("p")
        #sleep(1)
        if paragraphs is not None:
            for p in paragraphs:
                text += re.sub(html_pattern, '', p.text).strip() + ' '  
        return text
    except requests.exceptions.RequestException as e:
        msg = e

In [7]:
def save_dataset(q, json_object, date):
    total_count = 0
    for data in json_object:  
        try:
            data['text'] = get_news_data(data['url'])
            if data is None:
                continue
            if q not in os.listdir(f'./{TODAY}'):
                cwd = os.getcwd()
                os.mkdir(f'{cwd}/{TODAY}/{q}')
            total_count += 1
            file_error_symbols = []
            filename = f'{total_count}_{date.replace("/", "_")}'
            f = open(f'./{TODAY}/{q}//{filename}.json', "w")
            f.write(json.dumps(data, indent = 4))
            f.close()
        except requests.exceptions.RequestException as e:
            print(e)
    print(f'Saved {total_count} news data in {date}')
    #print(os.system("npx prettier -w ./dataset/*.json"))

In [8]:
def get_urls_from_file(filename):
    f = open(filename, "r")
    data = json.load(f)
    f.close()
    return data

In [9]:
def get_keywords_from_file(filename):
    keywords = []
    f = open(filename, "r")
    for keyword in f:
        keywords.append(keyword)
    f.close()
    return list(map(lambda keyword: keyword.strip(), keywords))

In [10]:
def parallel_save_dataset(q, urls):
    start = time.time()
    for url in urls:
        print(urls)
        save_dataset(q, urls[year], year)
    #Parallel(n_jobs = -1)(delayed(save_dataset)(q, urls[year], year) for year in urls)
    end = time.time()
    print('{:.2f} seconds used'.format(end - start))

In [11]:
#run all functions to save news data
def scrap_news_data(total, from_date, to_date, keywords_filename="keywords.txt"):
    #count used time
    start = time.time()
    #step 1 - load keywords file
    keywords = get_keywords_from_file(keywords_filename)
    for query in keywords:
        print(f'Scrapping {query} ...')
        #step 2 - save urls to json file
        save_news_urls(query, total, from_date, to_date)
        #step 3 - get urls object from previous saved json file
        urls = get_urls_from_file(f'./{TODAY}/{query}.json')
        #step 4 - web scrapping news data from specific year of urls & save into directory seperately
        save_dataset(query, urls, from_date)
        #parallel_save_dataset(query, urls, date)
        #save_dataset(query, urls)
        
    end = time.time()
    print('[DONE] {:.2f} seconds used'.format(end - start))

In [12]:
def json_to_csv(q, json_obj, index):
    text = None
    title = None
    date = None
    url = json_obj['url']
    if 'date' in json_obj.keys():
        date = json_obj['date']
    if json_obj['text'] is not None:
        text = re.sub(r'[,.\'"|]', '', json_obj['text'])
    if 'title' in json_obj.keys() and json_obj['title'] != None:
        title = re.sub(r'[,.\'"|]', '', json_obj['title'])
    if text != '' and text != None:
        text = ' '.join(text.encode('utf-8').decode().split())
    if title != '' and title != None:
        title = ' '.join(title.split()).strip() 
    f = open(f'./{TODAY}/{q}.csv', 'a', encoding='utf-8')
    f.write(f'{index},\"{title}\",\"{date}\",\"{url}\",\"{text}\"\n')
    f.close()

In [13]:
def to_date_time_format(date):
    date = date.split('/')
    date = list(map(lambda x: int(x), date))
    return datetime.datetime(date[2], date[1], date[0])

def to_google_param_format(date):
    return f'{date.month}/{date.day}/{date.year}'

def save_json_object(filename, json_object):
    f = open(f'./{TODAY}/{filename}', 'w')
    f.write(json.dumps(json_object, indent = 4))
    f.close()
    
def get_date_range_list(from_date, to_date):
    from_date_time = to_date_time_format(from_date)
    to_date_time = to_date_time_format(to_date)
    day_difference = (to_date_time - from_date_time).days
    datetime_list = [from_date_time + datetime.timedelta(days=x) for x in range(day_difference + 1)]
    date_range_list = list(map(lambda x: to_google_param_format(x), datetime_list))
    return date_range_list
    
def parallel_save_dataset(query, from_date, to_date):
    # count used time
    start = time.time()
    # JSON object
    json_object = dict()
    # get from date & to date difference days
    date_range_list = get_date_range_list(from_date, to_date)
    # iterate through date range list
    for date in date_range_list:
        urls = save_news_urls(query, total_news, date, date)
        json_object[date] = urls
        
    # replace date format to file savable characters
    filename = f'{query}.json'
    # temporary save of the scrapped urls
    save_json_object(filename, json_object)
    # parallel computing to save news data
    Parallel(n_jobs = -1)(delayed(save_dataset)(query, json_object[date], date) for date in json_object)
    #for date in json_object:
    #    save_dataset(query, json_object[date], date)
    # convert all json file in single csv file
    all_json_to_csv(stock_name)
    print(f'{filename}: [DONE]')

In [14]:
def all_json_to_csv(stock_name):
    cols = "title,date,url,text\n"
    keywords = get_keywords_from_file('keywords.txt')
    for keyword in keywords:
        f = open(f'./{TODAY}/{stock_name}.csv', 'w')
        f.write(cols)
        f.close()
        for index, filename in enumerate(os.listdir(f'./{TODAY}/{stock_name}')):
            if '.json' not in filename:
                continue
            json_file = open(f'./{TODAY}/{stock_name}/{filename}', 'r')
            json_obj = json.load(json_file)
            json_file.close()
            json_to_csv(stock_name, json_obj, index)
    print('Done')

In [15]:
def get_preprocessed_text(text):
    stop_words = set(stopwords.words("english"))
    special_characters = "!@#$%^&*()-+?_=,<>\""
    lemmatizer = WordNetLemmatizer()
    tokenized_words = word_tokenize(text.lower())
    lemmatized_words = [lemmatizer.lemmatize(word) for word in tokenized_words]
    preprocessed_words = [word for word in lemmatized_words if word not in stop_words]
    preprocessed_words = [word for word in lemmatized_words if word not in special_characters]
    preprocessed_text = " ".join(preprocessed_words)
    return preprocessed_text

In [16]:
def get_weighted_sentiment_score(news_df):
    sentiment_obj = dict()
    sentiment_score_df = []
    news_df_length = len(news_df)
    
    for i in range(news_df_length):
        data = news_df.iloc[i]
        date = data['date']
        title = data['title']
        text = data['text']
        preprocessed_text = get_preprocessed_text(text)
        sentiment_result = sentiment_pipe(preprocessed_text)[0]
        score = sentiment_result['score']
        label = sentiment_result['label']
        
        if date not in sentiment_obj.keys():
            sentiment_obj[date] = dict()
            sentiment_obj[date]['score'] = 0
            sentiment_obj[date]['count'] = 0
        if label != 'neutral': 
            if label == 'positive':
                sentiment_obj[date]['score'] += score
            else:
                sentiment_obj[date]['score'] -= score
            sentiment_obj[date]['count'] += 1
    
    for date in sentiment_obj:
        date_list = list(map(lambda x: int(x), date.split('/')))
        datetime_obj = datetime.datetime(date_list[2], date_list[0], date_list[1])
        score = sentiment_obj[date]['score']
        count = sentiment_obj[date]['count']
        if count > 0:
            item = [datetime_obj,  score / count]
            sentiment_score_df.append(item)
        
    sentiment_score_df = sorted(sentiment_score_df, key=lambda x: x[0])
    sentiment_score_df = list(map(lambda x: [str(x[0].date()), x[1]], sentiment_score_df))
    sentiment_score_df = pd.DataFrame(sentiment_score_df, columns =['Date', 'Score'], dtype = float) 
    return sentiment_score_df

In [17]:
def get_movement_accuracy(stock_df, sentiment_df):
    stock_available_dates = stock_df['Date'].values
    date_length = len(stock_available_dates)
    stock_val_list = stock_df['Adj Close'].values
    sentiment_date_list = sentiment_df['Date'].values
    
    real_label_list = []
    predict_label_list = []
    
    for i in range(len(sentiment_date_list)):
        if sentiment_date_list[i] not in stock_available_dates:
            sentiment_df = sentiment_df.drop(i)
    
    sentiment_score_list = sentiment_df['Score'].values

    for i in range(1, date_length):
        real_diff = stock_val_list[i] - stock_val_list[i-1]
        sentiment_diff = sentiment_score_list[i] - sentiment_score_list[i-1]
        
        if real_diff > 0:
            label = 'up'
        else:
            label = 'drop'
            
        real_label_list.append(label)
        
        if sentiment_diff > 0:
            label = 'up'
        else:
            label = 'drop'
            
        predict_label_list.append(label)
        
    # compare the movement accuracy
    label_list_length = len(real_label_list)
    total = 0
    for i in range(label_list_length):
        if real_label_list[i] == predict_label_list[i]:
            total += 1
    accuracy = total / date_length
    return accuracy

In [18]:
def plot_graph(index_arr, score_arr):
    fig, ax = plt.subplots(figsize=(16, 8))
    ax.plot(index_arr, score_arr)
    ax.set_title(f'AAPL Sentiment Score')
    ax.set_xlabel(f'Date')
    ax.set_ylabel(f'Score')
    ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.5))

In [19]:
parallel_save_dataset(stock_name, from_date, to_date)

Done
NVDA.json: [DONE]


In [20]:
news_df = pd.read_csv(f"./{TODAY}/{stock_name}.csv")
news_df = news_df.mask(news_df.eq('None')).dropna(subset=['text'])
news_df

,title,date,url,text
0,NaN,1/10/2024,https://investorplace.com/market360/2024/01/op...,Countdown to Luke Lango’s “2nd Trillion-Dollar...
1,The Datacenter GPU Gravy Train That No One Wil...,1/11/2024,https://www.nextplatform.com/2024/01/11/the-da...,We have five decades of very fine-grained anal...
2,Believe It Or Not Nvidia Stock Is Still Cheap ...,1/12/2024,https://seekingalpha.com/article/4662529-belie...,Antonio Bordunovi Antonio Bordunovi Nvidia Cor...
3,Forget Nvidia Heres a Better Magnificent Seven...,1/13/2024,https://www.fool.com/investing/2024/01/13/forg...,Founded in 1993 The Motley Fool is a financial...
5,NVIDIA Corp (NVDA)s Winning Formula: Financial...,1/15/2024,https://www.gurufocus.com/news/2153449/nvidia-...,NVIDIA Corp (NVDA Financial) has recently been...
...,...,...,...,...
1115,Should You Buy Nvidia After Its Huge Run? ETFs...,2/6/2024,https://www.zacks.com/stock/news/2221565/shoul...,We use cookies to understand how you use our s...
1116,AMD Broadcom Marvell Nvidia: Big gainers from ...,2/7/2024,https://www.marketbeat.com/originals/amd-broad...,A new Bank of America report found that data c...
1117,NaN,2/8/2024,https://www.investors.com/etfs-and-funds/secto...,BREAKING: Futures Fall Slightly Just when you ...
1118,Morning Brew: Nvidia Sets Up New AI Chip Unit ...,2/9/2024,https://www.gurufocus.com/news/2225804/morning...,Amid a rapidly evolving technological landscap...


In [21]:
stock_df = yf.download(stock_name, start=f'2024-01-01', end=f'2024-03-01')
stock_df = stock_df.reset_index()
stock_df['Date'] = stock_df['Date'].apply(lambda x: str(x.date()))

[*********************100%%**********************]  1 of 1 completed


In [ ]:
sentiment_df = get_weighted_sentiment_score(news_df)

Token indices sequence length is longer than the specified maximum sequence length for this model (1196 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
sentiment_df.to_csv(f'./{TODAY}/{stock_name}_sentiment.csv', index=False)

In [ ]:
accuracy = get_movement_accuracy(stock_df, sentiment_df)
print(f'{stock_name} Sentiment Analysis Accuracy: {accuracy}')